In [2]:
import pandas as pd
import numpy as np
import json
import math
import glob
import nltk



In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import SnowballStemmer
from scipy.spatial import distance
from matplotlib import pyplot as plt
from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize, word_tokenize

In [4]:
df = pd.read_csv('C:\\Users\\asus\\Covid Data\\metadata.csv')
doc_paths = 'C:\\Users\\asus\\Covid Data\\pdf_json.json'
df.sha.fillna("", inplace=True)

#get text for articles that are available


C:\Users\asus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [6]:
df.shape

(522159, 19)

In [7]:
def get_text(sha):
    if sha == "":
        return ""
    document_path = [x for x in doc_paths if sha in x]
    if not document_path:
        return ""
    with open(document_path[0]) as f:
        file = json.load(f)
        full_text = []
        #iterate over abstract and body part
        for part in ['abstract', 'body_text']:
            # iterate over each paragraph
            for text_part in file[part]:
                text = text_part['text']
                # remove citations from each paragraph
                for citation in text_part['cite_spans']:
                    text = text.replace(citation['text'], "")
                full_text.append(text)
            
        return str.join(' ', full_text)


In [8]:
%time df['text'] = df.apply(lambda x: get_text(x.sha), axis=1)

Wall time: 8.99 s


In [9]:
from nltk.stem import WordNetLemmatizer
analyzer = CountVectorizer().build_analyzer()
lemmatizer = WordNetLemmatizer()
def preprocess1(doc):
    doc=doc.lower()
    return str.join(" ", [lemmatizer.lemmatize(w) for w in analyzer(doc)])

def preprocess_row1(row):
    text = str.join(' ', [str(row.title), str(row.abstract), str(row.text)])
    return preprocess1(text)


In [10]:
%time df['lempreprocessed'] = df.apply(lambda x: preprocess_row1(x), axis=1)

Wall time: 9min 34s


In [46]:
cv = CountVectorizer(max_df=0.95, stop_words='english')
%time word_count = cv.fit_transform(df.lempreprocessed)
tfidf_tr = TfidfTransformer(smooth_idf=True, use_idf=True)
%time tfidf_tr.fit(word_count)

Wall time: 4min 17s
Wall time: 1.02 s


TfidfTransformer()

In [44]:
def get_word_vector(document):
    w_vector = tfidf_tr.transform(cv.transform([document]))
    return w_vector 

sample_tfidfvectorizer= get_word_vector(df.iloc[0].lempreprocessed)
df1 = pd.DataFrame(sample_tfidfvectorizer.T.todense(), index=cv.get_feature_names(), columns=["tfidf"]) 
df1.sort_values(by=["tfidf"],ascending=False)

In [13]:
%time df['word_vector'] = df.lempreprocessed.apply(get_word_vector)

Wall time: 1h 2min 21s


feature_names = cv.get_feature_names()
def get_words_with_value(w_vector):
    return sorted([(feature_names[ind], val) for ind, val in zip(w_vector.indices, w_vector.data)], key=lambda x: x[1], reverse=True)

In [14]:
def calculate_distance_between_words_vectors(search_words_indices, search_vec, document_vector):
    document_vec = document_vector[0, search_words_indices].toarray()
    return distance.euclidean([search_vec], document_vec)

def get_rel_doc
search_vector = get_word_vector(preprocess(topic))
search_words_indices = search_vector.indices
search_vec = search_vector.data
distance_idx = df.apply(lambda x: calculate_distance_between_words_vectors(search_words_indices, search_vec, x.word_vector), axis=1)
relevant_indexes = distance_idx.sort_values().head(10).index
result_columns = ["title", "doi", "pmcid", "license", "authors"]
result = df[result_columns].iloc[relevant_indexes].fillna("")
return result  


In [15]:
def display_friendly_results(df_result):
    display_columns = [
 'title',
 'doi',
 'abstract','authors',
 'lempreprocessed',
 ]
    display(df_result[display_columns].reset_index(drop=True))


In [27]:
def get_related_documents(text, number_of_documents):    
    search_vector = get_word_vector(preprocess1(text))
    search_words_indices = search_vector.indices
    search_vec = search_vector[0, search_words_indices].toarray()
    distance_idx = df.apply(lambda x: calculate_distance_between_words_vectors(search_words_indices, search_vec, x.word_vector), axis=1)
    relevant_indexes = distance_idx.sort_values().head(number_of_documents).index 
    result_columns = [
 'title',
 'doi',
 'abstract','authors',
 'lempreprocessed',
]
    result = df[result_columns].iloc[relevant_indexes].fillna("")
    return result

In [23]:
topics = {
    "What is known about transmission, incubation, and environmental stability?" :
    [
        "Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.",
        "Prevalence of asymptomatic shedding and transmission (e.g., particularly children).",
       
    ],
    "What do we know about COVID-19 risk factors?":
    [
        "Data on potential risks factors",
        
        "Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors",
        
    ],
    "What do we know about virus genetics, origin, and evolution?":
    [
        "Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time.",
        "Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged.",
        "Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over.",
        "Evidence of whether farmers are infected, and whether farmers could have played a role in the origin.",
        "Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia.",
        "Experimental infections to test host range for this pathogen.",
        "Animal host(s) and any evidence of continued spill-over to humans",
        "Socioeconomic and behavioral risk factors for this spill-over",
        "Sustainable risk reduction strategies"
    ],
    "What do we know about vaccines and therapeutics?":
    [
        "Effectiveness of drugs being developed and tried to treat COVID-19 patients.",
        "Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.",
        "Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.",
        "Exploration of use of best animal models and their predictive value for a human vaccine.",
        "Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.",
        "Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need.",
        "Efforts targeted at a universal coronavirus vaccine.",
        "Efforts to develop animal models and standardize challenge studies",
        "Efforts to develop prophylaxis clinical studies and prioritize in healthcare workers",
        "Approaches to evaluate risk for enhanced disease after vaccination",
        "Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models [in conjunction with therapeutics],",
    ],
    "What do we know about diagnostics and surveillance?":
    [
        "How widespread current exposure is to be able to make immediate policy recommendations on mitigation measures. Denominators for testing and a mechanism for rapidly sharing that information, including demographics, to the extent possible. Sampling methods to determine asymptomatic disease (e.g., use of serosurveys (such as convalescent samples) and early detection of disease (e.g., use of screening of neutralizing antibodies such as ELISAs).",
        "Efforts to increase capacity on existing diagnostic platforms and tap into existing surveillance platforms.",
        "Recruitment, support, and coordination of local expertise and capacity (public, private—commercial, and non-profit, including academic), including legal, ethical, communications, and operational issues.",
        "National guidance and guidelines about best practices to states (e.g., how states might leverage universities and private laboratories for testing purposes, communications to public health officials and the public).",
        "Development of a point-of-care test (like a rapid influenza test) and rapid bed-side tests, recognizing the tradeoffs between speed, accessibility, and accuracy.",
        "Rapid design and execution of targeted surveillance experiments calling for all potential testers using PCR in a defined area to start testing and report to a specific entity. These experiments could aid in collecting longitudinal samples, which are critical to understanding the impact of ad hoc local interventions (which also need to be recorded).",
        "Separation of assay development issues from instruments, and the role of the private sector to help quickly migrate assays onto those devices.",
        "Efforts to track the evolution of the virus (i.e., genetic drift or mutations) and avoid locking into specific reagents and surveillance/detection schemes.",
        "Latency issues and when there is sufficient viral load to detect the pathogen, and understanding of what is needed in terms of biological and environmental sampling.",
        "Use of diagnostics such as host response markers (e.g., cytokines) to detect early disease or predict severe disease progression, which would be important to understanding best clinical practice and efficacy of therapeutic interventions.",
        "Policies and protocols for screening and testing.",
        "Policies to mitigate the effects on supplies associated with mass testing, including swabs and reagents.",
        "Technology roadmap for diagnostics.",
        "Barriers to developing and scaling up new diagnostic tests (e.g., market forces), how future coalition and accelerator models (e.g., Coalition for Epidemic Preparedness Innovations) could provide critical funding for diagnostics, and opportunities for a streamlined regulatory environment.",
        "New platforms and technology (e.g., CRISPR) to improve response times and employ more holistic approaches to COVID-19 and future diseases.",
        "Coupling genomics and diagnostic testing on a large scale.",
        "Enhance capabilities for rapid sequencing and bioinformatics to target regions of the genome that will allow specificity for a particular variant.",
        "Enhance capacity (people, technology, data) for sequencing with advanced analytics for unknown pathogens, and explore capabilities for distinguishing naturally-occurring pathogens from intentional.",
        "One Health surveillance of humans and potential sources of future spillover or ongoing exposure for this organism and future pathogens, including both evolutionary hosts (e.g., bats) and transmission hosts (e.g., heavily trafficked and farmed wildlife and domestic food and companion species), inclusive of environmental, demographic, and occupational risk factors."
    ],
    "What do we know about non-pharmaceutical interventions?":
    [
        "Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases.",
        "Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments.",
        "Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches.",
        "Methods to control the spread in communities, barriers to compliance and how these vary among different populations.",
        "Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status.",
        "Policy changes necessary to enable the compliance of individuals with limited resources and the underserved with NPIs.",
        "Research on why people fail to comply with public health advice, even if they want to do so (e.g., social or financial costs may be too high).",
        "Research on the economic impact of this or any pandemic. This would include identifying policy and programmatic alternatives that lessen/mitigate risks to critical government services, food distribution and supplies, access to critical household supplies, and access to health diagnoses, treatment, and needed care, regardless of ability to pay."
    ],
    "What has been published about medical care?":
    [
        "Resources to support skilled nursing facilities and long term care facilities.",
        "Mobilization of surge medical staff to address shortages in overwhelmed communities",
        "Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies",
        "Extracorporeal membrane oxygenation (ECMO) outcomes data of COVID-19 patients",
        "Outcomes data for COVID-19 after mechanical ventilation adjusted for age.",
        "Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest.",
        "Application of regulatory standards (e.g., EUA, CLIA) and ability to adapt care to crisis standards of care level.",
        "Approaches for encouraging and facilitating the production of elastomeric respirators, which can save thousands of N95 masks.",
        "Best telemedicine practices, barriers and faciitators, and specific actions to remove/expand them within and across state boundaries.",
        "Guidance on the simple things people can do at home to take care of sick people and manage disease.",
        "Oral medications that might potentially work.",
        "Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually.",
        "Best practices and critical challenges and innovative solutions and technologies in hospital flow and organization, workforce protection, workforce allocation, community-based support resources, payment, and supply chain management to enhance capacity, efficiency, and outcomes.",
        "Efforts to define the natural history of disease to inform clinical care, public health interventions, infection prevention control, transmission, and clinical trials",
        "Efforts to develop a core clinical outcome set to maximize usability of data across a range of trials",
        "Efforts to determine adjunctive and supportive interventions that can improve the clinical outcomes of infected patients (e.g. steroids, high flow oxygen)"
    ],
    "What has been published about information sharing and inter-sectoral collaboration?":
    [
        "Methods for coordinating data-gathering with standardized nomenclature.",
        "Sharing response information among planners, providers, and others.",
        "Understanding and mitigating barriers to information-sharing.",
        "How to recruit, support, and coordinate local (non-Federal) expertise and capacity relevant to public health emergency response (public, private, commercial and non-profit, including academic).",
        "Integration of federal/state/local public health surveillance systems.",
        "Value of investments in baseline public health response infrastructure preparedness",
        "Modes of communicating with target high-risk populations (elderly, health care workers).",
        "Risk communication and guidelines that are easy to understand and follow (include targeting at risk populations’ families too).",
        "Communication that indicates potential risk of disease to all population groups.",
        "Misunderstanding around containment and mitigation.",
        "Action plan to mitigate gaps and problems of inequity in the Nation’s public health capability, capacity, and funding to ensure all citizens in need are supported and can access information, surveillance, and treatment.",
        "Measures to reach marginalized and disadvantaged populations.",
        "Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities.",
        "Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment.",
        "Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care"
    ],
    "What has been published about ethical and social science considerations?":
    [
        "Efforts to articulate and translate existing ethical principles and standards to salient issues in COVID-2019",
        "Efforts to embed ethics across all thematic areas, engage with novel ethical issues that arise and coordinate to minimize duplication of oversight",
        "Efforts to support sustained education, access, and capacity building in the area of ethics",
        "Efforts to establish a team at WHO that will be integrated within multidisciplinary research and operational platforms and that will connect with existing and expanded global networks of social sciences.",
        "Efforts to develop qualitative assessment frameworks to systematically collect information related to local barriers and enablers for the uptake and adherence to public health measures for prevention and control. This includes the rapid identification of the secondary impacts of these measures. (e.g. use of surgical masks, modification of health seeking behaviors for SRH, school closures)",
        "Efforts to identify how the burden of responding to the outbreak and implementing public health measures affects the physical and psychological health of those providing care for Covid-19 patients and identify the immediate needs that must be addressed.",
        "Efforts to identify the underlying drivers of fear, anxiety and stigma that fuel misinformation and rumor, particularly through social media.",
    ]
}

In [24]:
l=[]

In [25]:
def display_topics_results(question):
    for topic in topics[question]:
        res=get_related_documents(topic,6)
        for i in res.index:
            l.append(res['lempreprocessed'][i])
        display_friendly_results(res)

In [29]:
%time display_topics_results("What is known about transmission, incubation, and environmental stability?")

,title,doi,abstract,authors,lempreprocessed
0,Delivering urgent urological surgery during th...,,"Since first reported in December 2019, the nov...","Paramore, Louise; Yang, Bob; Abdelmotagly, Yeh...",delivering urgent urological surgery during th...
1,Delivering urgent urological surgery during th...,10.1111/bju.15110,"Since first reported in December 2019, the nov...","Paramore, Louise; Yang, Bob; Abdelmotagly, Yeh...",delivering urgent urological surgery during th...
2,Association between the Severity of Influenza ...,10.1371/journal.pone.0148506,"BACKGROUND: In early 2013, a novel avian-origi...","Virlogeux, Victor; Yang, Juan; Fang, Vicky J.;...",association between the severity of influenza ...
3,The new coronavirus-cОvid-19 in Uzbekistan,,The article includes the latest researches on ...,"Matnazarova, Gulbahor; Mirtazaev, Omonturdi; B...",the new coronavirus cоvid 19 in uzbekistan the...
4,Pre-symptomatic transmission of novel coronavi...,,We used contact tracing to document how COVID-...,"Kong, Dechuan; Zheng, Yang; Wu, Huanyu; Pan, H...",pre symptomatic transmission of novel coronavi...
5,Pre‐symptomatic transmission of novel coronavi...,10.1111/irv.12773,We used contact tracing to document how COVID‐...,"Kong, Dechuan; Zheng, Yang; Wu, Huanyu; Pan, H...",pre symptomatic transmission of novel coronavi...


,title,doi,abstract,authors,lempreprocessed
0,SARS-CoV-2 Point Prevalence among Asymptomatic...,10.1093/jpids/piaa102,Asymptomatic SARS-CoV-2 carriage among hospita...,"Patel, Ami B; Clifford, Andrea; Creaden, Julie...",sars cov point prevalence among asymptomatic h...
1,SARS-CoV-2 Point Prevalence among Asymptomatic...,,Asymptomatic SARS-CoV-2 carriage among hospita...,"Patel, Ami B; Clifford, Andrea; Creaden, Julie...",sars cov point prevalence among asymptomatic h...
2,Shedding of coronavirus‐like particles by chil...,10.1002/jmv.1890270219,Stools from 266 children in four districts of ...,"Kidd, A. H.; Esrey, S. A.; Ujfalusi, Maria J.",shedding of coronavirus like particle by child...
3,Viral RNA Shedding and Transmission Potential ...,,We studied the pattern and duration of viral R...,"Samaddar, Arghadip Gadepalli Ravisekhar Nag Vi...",viral rna shedding and transmission potential ...
4,Viral RNA Shedding and Transmission Potential ...,10.1093/ofid/ofaa599,We studied the pattern and duration of viral R...,"Samaddar, Arghadip; Gadepalli, Ravisekhar; Nag...",viral rna shedding and transmission potential ...
5,Viral Ribonucleic Acid Shedding and Transmissi...,,We studied the pattern and duration of viral r...,"Samaddar, A.; Gadepalli, R.; Nag, V. L.; Misra...",viral ribonucleic acid shedding and transmissi...


Wall time: 34min


In [32]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch

In [35]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [36]:
question = "What is known about transmission, incubation, and environmental stability"

In [37]:
def info(abstract,question):
    paragraph = abstract
    encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special=True)
    inputs = encoding['input_ids']  #Token embeddings
    sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
    start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    if end_index>start_index:
        answer = tokens[start_index:end_index]
        
    else:
        answer = tokens[start_index:]
    return answer

In [39]:
a=[]


In [40]:
for i in list(set(l)):
    print(len(i))
    answer=info(i,question)
    a.append(answer)
    

893


Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


424


Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


1379


...

473


Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


493


Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


1170


In [41]:
for i in a:
    corrected_answer = ''
    for word in i:
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word
    print(len(corrected_answer),corrected_answer)
    

82  [CLS] what is known about transmission , incubation , and environmental stability
35  risk of transmission to healthcare
1474  [CLS] what is known about transmission , incubation , and environmental stability [SEP] association between the severity of influenza h7n9 virus infection and length of the incubation period background in early 2013 novel avian origin influenza h7n9 virus emerged in china and ha caused sporadic human infection the incubation period is the delay from infection until onset of symptom and varies from person to person few previous study have examined whether the duration of the incubation period correlate with subsequent disease severity method and finding we analyzed data of period of exposure on 395 human case of laboratory confirmed influenza h7n9 virus infection in china in bayesian framework using weibull distribution we found longer incubation period for the 173 fatal case with mean of day 95 credibility interval cri compared to mean of day 95 cri for the